# Capstone Project for IBM's Data Science Specialization course.  

In [37]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [38]:
ColumnHeads = ['ObjectID', 'Zip', 'Town', 'County', 'Latitude', 'Longitude']
df1=pd.read_csv(r"MOCO.csv", names=ColumnHeads)
df2 = df1[['Zip', 'Town', 'County', 'Latitude','Longitude']]
df2.head()

,Zip,Town,County,Latitude,Longitude
0,20812,GLEN ECHO,Montgomery,38.971674,-77.143039
1,20816,BETHESDA,Montgomery,38.968822,-77.139069
2,20818,CABIN JOHN,Montgomery,38.984851,-77.158608
3,20894,BETHESDA,Montgomery,38.993788,-77.096285
4,20892,BETHESDA,Montgomery,38.995341,-77.096464


In [39]:
df2.shape

(97, 5)

In [40]:
df2.head()

,Zip,Town,County,Latitude,Longitude
0,20812,GLEN ECHO,Montgomery,38.971674,-77.143039
1,20816,BETHESDA,Montgomery,38.968822,-77.139069
2,20818,CABIN JOHN,Montgomery,38.984851,-77.158608
3,20894,BETHESDA,Montgomery,38.993788,-77.096285
4,20892,BETHESDA,Montgomery,38.995341,-77.096464


# Concatenating the longitude and latitude data to the county 

### Using Nominatim as geolocator

In [41]:
address = 'Montgomery County, MD'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Montgomery County are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Montgomery County are 39.1406267, -77.2075612.


## Show the county and location of each town in Montgomery County

In [42]:
# create map of Montgomery County, MD using latitude and longitude values
map_MOCO = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, county, town in zip(df2['Latitude'], df2['Longitude'], df2['County'], df2['Town']):
    label = '{}, {}'.format(town, county)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3199cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_MOCO)  
    
map_MOCO

### Set up access to Foursquare

In [43]:
CLIENT_ID = 'ZMOS5JBCPGLY0IJ1WYBR1SHNIV2DP2SMRVIEZHLF3IZ5U22M' # your Foursquare ID
CLIENT_SECRET = 'O3K0YWMF535K5VJWICWUDAJZXPFQ0C5REELKD1TTTLGG320F' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZMOS5JBCPGLY0IJ1WYBR1SHNIV2DP2SMRVIEZHLF3IZ5U22M
CLIENT_SECRET:O3K0YWMF535K5VJWICWUDAJZXPFQ0C5REELKD1TTTLGG320F


In [44]:
address = 'Montgomery County, MD'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Montgomery County are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Montgomery County are 39.1406267, -77.2075612.


### Rockville

In [45]:
df3 = df2[df2['Town'].str.contains('ROCKVILLE')]
df3.head()
MOCO_data = df3.reset_index(drop=True)
MOCO_data.head()

,Zip,Town,County,Latitude,Longitude
0,20852,ROCKVILLE,Montgomery,39.080496,-77.141398
1,20852,ROCKVILLE,Montgomery,39.080529,-77.141561
2,20852,ROCKVILLE,Montgomery,39.081572,-77.145846
3,20851,ROCKVILLE,Montgomery,39.093381,-77.119259
4,20850,ROCKVILLE,Montgomery,39.120674,-77.174714


### Generate a new map with only Rockville

In [46]:

map_town = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, county, town in zip(MOCO_data['Latitude'], MOCO_data['Longitude'], MOCO_data['County'], MOCO_data['Town']):
#for lat, lng, town in zip(MOCO_data['Latitude'], MOCO_data['Longitude'], MOCO_data['Town']):
   label = '{}, {}'.format(town, county)
   label = folium.Popup(label, parse_html=True)
   folium.CircleMarker(
       [lat, lng],
       radius=3,
       popup=label,
       color='green',
       fill=True,
       fill_color='#3199cc',
       fill_opacity=0.3,
       parse_html=False).add_to(map_town)  
    
map_town

### Find the top 100 venues in Rockville

In [47]:
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    town_latitude, 
    town_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=ZMOS5JBCPGLY0IJ1WYBR1SHNIV2DP2SMRVIEZHLF3IZ5U22M&client_secret=O3K0YWMF535K5VJWICWUDAJZXPFQ0C5REELKD1TTTLGG320F&v=20180605&ll=39.1206742,-77.17471395&radius=500&limit=100'

### Find out what types of businesses are in town

In [48]:
results = requests.get(url).json()

In [49]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [50]:
# clean up the Json
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean up
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(5)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,name,categories,lat,lng
0,Sheeshababas House of Hookah,Hookah Bar,39.120685,-77.178042
1,King Farm Dog Park,Dog Run,39.119046,-77.177848
2,Dunkin',Donut Shop,39.121463,-77.177326
3,Big Wang's Cuicine,Asian Restaurant,39.118683,-77.170720
4,Checkers,Fast Food Restaurant,39.120858,-77.178897


### Find out how many were identified

In [51]:
print('{} venues were returned'.format(nearby_venues.shape[0]))

10 venues were returned


# Now we'll explore all towns in Montgomery County

In [52]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Town', 
                  'Town Latitude', 
                  'Town Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [53]:
MOCO_venues = getNearbyVenues(names=MOCO_data['Town'],
                                   latitudes=MOCO_data['Latitude'],
                                   longitudes=MOCO_data['Longitude']
                                  )

MOCO_venues

ROCKVILLE
ROCKVILLE
ROCKVILLE
ROCKVILLE
ROCKVILLE
ROCKVILLE


,Town,Town Latitude,Town Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ROCKVILLE,39.080496,-77.141398,Earth Treks Climbing Center,39.078518,-77.143439,Gym / Fitness Center
1,ROCKVILLE,39.080496,-77.141398,Marlo Furniture,39.079261,-77.143308,Furniture / Home Store
2,ROCKVILLE,39.080496,-77.141398,La Limeña,39.077191,-77.141612,Peruvian Restaurant
3,ROCKVILLE,39.080496,-77.141398,El Mariachi,39.077224,-77.141474,Mexican Restaurant
4,ROCKVILLE,39.080496,-77.141398,Kiin Imm,39.076970,-77.140792,Thai Restaurant
5,ROCKVILLE,39.080496,-77.141398,Yuan Fu Vegetarian,39.078382,-77.140439,Vegetarian / Vegan Restaurant
6,ROCKVILLE,39.080496,-77.141398,Jumbo Jumbo Bubble Tea Express,39.077343,-77.141623,Bubble Tea Shop
7,ROCKVILLE,39.080496,-77.141398,Roy Rogers,39.079410,-77.141845,Fast Food Restaurant
8,ROCKVILLE,39.080496,-77.141398,OM Fine Indian Cuisine,39.077266,-77.141651,Indian Restaurant
9,ROCKVILLE,39.080496,-77.141398,Enterprise Rent-A-Car,39.080137,-77.142919,Rental Car Location


### Let's check to see how big the dataframe is

In [54]:
print(MOCO_venues.shape)
MOCO_venues.head()

(66, 7)


,Town,Town Latitude,Town Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ROCKVILLE,39.080496,-77.141398,Earth Treks Climbing Center,39.078518,-77.143439,Gym / Fitness Center
1,ROCKVILLE,39.080496,-77.141398,Marlo Furniture,39.079261,-77.143308,Furniture / Home Store
2,ROCKVILLE,39.080496,-77.141398,La Limeña,39.077191,-77.141612,Peruvian Restaurant
3,ROCKVILLE,39.080496,-77.141398,El Mariachi,39.077224,-77.141474,Mexican Restaurant
4,ROCKVILLE,39.080496,-77.141398,Kiin Imm,39.076970,-77.140792,Thai Restaurant


In [55]:
MOCO_venues.groupby('Town').count()

,Town Latitude,Town Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Town,,,,,,
ROCKVILLE,66,66,66,66,66,66


In [56]:
print('There are {} unique categories'.format(len(MOCO_venues['Venue Category'].unique())))

There are 37 unique categories


In [57]:
# one hot encoding
MOCO_onehot = pd.get_dummies(MOCO_venues[['Venue Category']], prefix="", prefix_sep="")

# add town column back to dataframe
MOCO_onehot['Town'] = MOCO_venues['Town'] 

# move neighborhood column to the first column
fixed_columns = [MOCO_onehot.columns[-1]] + list(MOCO_onehot.columns[:-1])
MOCO_onehot = MOCO_onehot[fixed_columns]

MOCO_onehot.head()

,Town,ATM,Asian Restaurant,Athletics & Sports,Automotive Shop,Bank,Baseball Field,Beer Bar,Breakfast Spot,Bubble Tea Shop,Café,Convenience Store,Deli / Bodega,Dog Run,Donut Shop,Farmers Market,Fast Food Restaurant,Furniture / Home Store,Gym,Gym / Fitness Center,Hookah Bar,Ice Cream Shop,Indian Restaurant,Latin American Restaurant,Metro Station,Mexican Restaurant,Movie Theater,Noodle House,Outdoors & Recreation,Park,Peruvian Restaurant,Pizza Place,Pool,Rental Car Location,Sandwich Place,Sushi Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant
0,ROCKVILLE,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,ROCKVILLE,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,ROCKVILLE,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,ROCKVILLE,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,ROCKVILLE,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [58]:
MOCO_onehot.shape

(66, 38)

### Group by town and by the mean of the frequency of each occurence

In [59]:
MOCO_grouped = MOCO_onehot.groupby('Town').mean().reset_index()
MOCO_grouped

,Town,ATM,Asian Restaurant,Athletics & Sports,Automotive Shop,Bank,Baseball Field,Beer Bar,Breakfast Spot,Bubble Tea Shop,Café,Convenience Store,Deli / Bodega,Dog Run,Donut Shop,Farmers Market,Fast Food Restaurant,Furniture / Home Store,Gym,Gym / Fitness Center,Hookah Bar,Ice Cream Shop,Indian Restaurant,Latin American Restaurant,Metro Station,Mexican Restaurant,Movie Theater,Noodle House,Outdoors & Recreation,Park,Peruvian Restaurant,Pizza Place,Pool,Rental Car Location,Sandwich Place,Sushi Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant
0,ROCKVILLE,0.015152,0.015152,0.030303,0.045455,0.030303,0.015152,0.015152,0.030303,0.030303,0.015152,0.030303,0.015152,0.015152,0.015152,0.015152,0.090909,0.045455,0.015152,0.045455,0.015152,0.015152,0.045455,0.015152,0.015152,0.060606,0.015152,0.030303,0.015152,0.015152,0.030303,0.030303,0.015152,0.060606,0.015152,0.015152,0.030303,0.030303


### Let's check the size of the new set

In [60]:
MOCO_grouped.shape

(1, 38)

### Print the top 3 venues for each town

In [61]:
num_top_venues = 3

for town in MOCO_grouped['Town']:
    print("----"+town+"----")
    temp = MOCO_grouped[MOCO_grouped['Town'] == town].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ROCKVILLE----
                  venue  freq
0  Fast Food Restaurant  0.09
1   Rental Car Location  0.06
2    Mexican Restaurant  0.06




### Now define function to sort the venues in descending order

In [62]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Create dataframe and display top 10 venues

In [63]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Town']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
towns_venues_sorted = pd.DataFrame(columns=columns)
towns_venues_sorted['Town'] = MOCO_grouped['Town']

for ind in np.arange(MOCO_grouped.shape[0]):
    towns_venues_sorted.iloc[ind, 1:] = return_most_common_venues(MOCO_grouped.iloc[ind, :], num_top_venues)

towns_venues_sorted.head()

,Town,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ROCKVILLE,Fast Food Restaurant,Rental Car Location,Mexican Restaurant,Gym / Fitness Center,Indian Restaurant,Automotive Shop,Furniture / Home Store,Athletics & Sports,Bank,Breakfast Spot


### Use k-means to segment this set of data into 5 clusters

In [64]:
# set number of clusters
kclusters = 1

MOCO_grouped_clustering = MOCO_grouped.drop('Town', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(MOCO_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:1] 

array([0], dtype=int32)

In [65]:
MOCO_merged = MOCO_data

# merge MOCO_grouped with MOCO_data to add latitude/longitude for each town
MOCO_merged = MOCO_merged.join(towns_venues_sorted.set_index('Town'), on='Town')

MOCO_merged.head() 

,Zip,Town,County,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,20852,ROCKVILLE,Montgomery,39.080496,-77.141398,Fast Food Restaurant,Rental Car Location,Mexican Restaurant,Gym / Fitness Center,Indian Restaurant,Automotive Shop,Furniture / Home Store,Athletics & Sports,Bank,Breakfast Spot
1,20852,ROCKVILLE,Montgomery,39.080529,-77.141561,Fast Food Restaurant,Rental Car Location,Mexican Restaurant,Gym / Fitness Center,Indian Restaurant,Automotive Shop,Furniture / Home Store,Athletics & Sports,Bank,Breakfast Spot
2,20852,ROCKVILLE,Montgomery,39.081572,-77.145846,Fast Food Restaurant,Rental Car Location,Mexican Restaurant,Gym / Fitness Center,Indian Restaurant,Automotive Shop,Furniture / Home Store,Athletics & Sports,Bank,Breakfast Spot
3,20851,ROCKVILLE,Montgomery,39.093381,-77.119259,Fast Food Restaurant,Rental Car Location,Mexican Restaurant,Gym / Fitness Center,Indian Restaurant,Automotive Shop,Furniture / Home Store,Athletics & Sports,Bank,Breakfast Spot
4,20850,ROCKVILLE,Montgomery,39.120674,-77.174714,Fast Food Restaurant,Rental Car Location,Mexican Restaurant,Gym / Fitness Center,Indian Restaurant,Automotive Shop,Furniture / Home Store,Athletics & Sports,Bank,Breakfast Spot


### Let's see this on a map

In [66]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(MOCO_merged['Latitude'],MOCO_merged['Longitude'], MOCO_merged['Town'], MOCO_merged['Cluster Labels']):
  
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

KeyError: 'Cluster Labels'

# Let's examine the data for each of the 5 clusters

### Cluster 1

In [ ]:
MOCO_merged.loc[MOCO_merged['Cluster Labels'] == 0, MOCO_merged.columns[[1] + list(range(5, MOCO_merged.shape[1]))]]

### Cluster 2

In [ ]:
MOCO_merged.loc[MOCO_merged['Cluster Labels'] == 1, MOCO_merged.columns[[1] + list(range(5, MOCO_merged.shape[1]))]]

### Cluster 3

In [ ]:
MOCO_merged.loc[MOCO_merged['Cluster Labels'] == 2, MOCO_merged.columns[[1] + list(range(5, MOCO_merged.shape[1]))]]

### Cluster 4

In [ ]:
MOCO_merged.loc[MOCO_merged['Cluster Labels'] == 3, MOCO_merged.columns[[1] + list(range(5, MOCO_merged.shape[1]))]]

### Cluster 5

In [ ]:
MOCO_merged.loc[MOCO_merged['Cluster Labels'] == 4, MOCO_merged.columns[[1] + list(range(5, MOCO_merged.shape[1]))]]